# Hasse Search Space Estimation

## Load Data

In [7]:
import json
from os import listdir
from pprint import pprint
from typing import List, Dict

In [9]:
# Load data

def get_file_paths(dir_path: str = "results") -> List[str]:
    file_names = listdir(dir_path)
    file_names = [file_name for file_name in file_names if file_name.endswith(".json")]

    file_paths = [
        f"{dir_path}/{file_name}"  for file_name in file_names
    ]
    return file_paths


def load_data(file_path: str) -> Dict:
    with open(file_path, "r") as data_file:
        return json.load(data_file)

experiment_paths = get_file_paths()
experiments = [
    load_data(file_path) for file_path in experiment_paths
]

# TODO: Remove
experiments = [
    experiment for experiment in experiments if experiment["config"]["gate_count"] <= 10
]

print(f"Experiment count: {len(experiments)}")
print(experiments[0])

Experiment count: 8
{'meta': {'start': '2025-11-14 14:29:53.932920', 'end': '2025-11-14 14:47:01.523753'}, 'config': {'gate_set': '[H, S, T, CX]', 'gate_count': 10, 'qubit_num': 3, 'sample_num': 100000, 'seed_num': 30}, 'results': {'simulation_times': [7.47153, 7.689302, 7.504083, 7.555472, 7.499643, 7.53596, 7.521314, 7.594106, 7.462234, 7.51527, 7.454071, 7.453916, 7.533094, 7.495804, 7.444111, 7.493087, 7.452408, 7.511089, 7.512671, 7.57489, 7.530768, 7.485159, 7.476999, 7.506042, 7.460574, 7.513438, 7.47497, 7.520497, 7.435024, 7.467667], 'mean_simulation_time': 7.504839766666667, 'hasse_duplicate_counts': [136, 144, 136, 152, 164, 159, 154, 142, 165, 157, 132, 141, 130, 137, 144, 139, 158, 133, 147, 141, 152, 146, 185, 152, 140, 142, 143, 164, 147, 149], 'mean_hasse_duplicate_count': 147.7, 'hasse_encoding_times': [24.409497, 24.561948, 24.488765, 24.511919, 24.443124, 24.490618, 24.39568, 24.480262, 24.403193, 24.364116, 24.388311, 24.347988, 24.39826, 24.410229, 24.466509, 24.32

In [ ]:
# Group experiments by qubit num

In [ ]:
# Group experimentes by gate count

## Compute Search Space Size

In [ ]:
# Calculate search space size

In [ ]:
# test with 